In [2]:

# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

### Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/
Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs).
Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given).
You may be inspired by the function `extract_numbers()` in [Code Snippet 4], or you can write your own function to "parse" a string.
Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (`Date.dt.dayofweek()==4`). You should see 32 records in total, 25 of it is not null.

(additional: you can read about [S-1 IPO filing](https://www.dfinsolutions.com/knowledge-hub/thought-leadership/knowledge-resources/what-s-1-ipo-filing) to understand the context)

In [8]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

C:\Users\kiramishima\AppData\Local\Temp\ipykernel_25652\2819786403.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  ipo_dfs = pd.read_html(response.text)


In [9]:
ipos_2023 = ipo_dfs[0]
ipos_2023.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [10]:
ipos_2023['Filing Date'] = pd.to_datetime(ipos_2023['Filing Date'], format='%b %d, %Y')

In [17]:
ipos_2023['Shares Offered'].replace({'-': np.nan}, inplace=True)
ipos_2023['Shares Offered'] = ipos_2023['Shares Offered'].astype(np.float64)

In [19]:
import re

# Code Snippet 4
def extract_numbers(input_string):
    split_string = input_string.split(" - ")
    y_match = re.search(r'(\d+.\d)', split_string[0])
    if len(split_string) > 1:
      m_match = re.search(r'(\d+.\d)', split_string[1])
      y1_number, m1_number = float(y_match.group(1)) if y_match else 0, float(m_match.group(1)) if m_match else 0
      return (y1_number + m1_number)/len(split_string)
    else:
      y0 = float(y_match.group(1)) if y_match else 0
      return y0

In [20]:
ipos_2023['Avg_price'] = ipos_2023['Price Range'].apply(lambda x: extract_numbers(x))

In [22]:
ipos_2023['Shares_offered_value'] = ipos_2023['Shares Offered'] * ipos_2023['Avg_price']

In [23]:
ipos_2023.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,0.0,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,0.0,NaN


In [28]:
ipos_filter = ipos_2023[(ipos_2023['Filing Date'].dt.dayofweek == 4)  & (ipos_2023['Filing Date'].dt.year == 2023)]

In [30]:
ipos_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 50 to 165
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           32 non-null     datetime64[ns]
 1   Symbol                32 non-null     object        
 2   Company Name          32 non-null     object        
 3   Price Range           32 non-null     object        
 4   Shares Offered        25 non-null     float64       
 5   Avg_price             32 non-null     float64       
 6   Shares_offered_value  25 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 2.0+ KB


In [31]:
total = ipos_filter['Shares_offered_value'].sum()
f"Total sum in millions: ${round(total/1000000)}M"

'total sum in millions: $286M'

### Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**


Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). 
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance. 

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this [link](https://stockanalysis.com/actions/changes/).
Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest. 

HINTs:
* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.


Additional: 
* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [45]:
from io import StringIO
import pandas as pd
import requests

def get_ipo_data(year):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    }
    url = f'https://stockanalysis.com/ipos/{year}/'
    print(f'Getting - {url}')
    response = requests.get(url, headers=headers)
    html_io = StringIO(response.text)
    print('Finishing')
    return pd.read_html(html_io)

In [46]:
# 2023
ipo_dfs = get_ipo_data(2023)

ipos_2023 = ipo_dfs[0]
ipos_2023.info()

Getting - https://stockanalysis.com/ipos/2023/
Finishing
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [48]:
# 2024
ipo_dfs = get_ipo_data(2024)

ipos_2024 = ipo_dfs[0]
ipos_2024.info()

Getting - https://stockanalysis.com/ipos/2024/
Finishing
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [50]:
df_collect = []
for year in range(2023, 2025):
    ipo_dfs = get_ipo_data(year)

    df_collect.append(ipo_dfs[0])

df_ipos = pd.concat(df_collect, ignore_index=True)

Getting - https://stockanalysis.com/ipos/2023/
Finishing
Getting - https://stockanalysis.com/ipos/2024/
Finishing


In [51]:
df_ipos['IPO Date'] = pd.to_datetime(df_ipos['IPO Date'], format="%b %d, %Y")

In [52]:
df_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      218 non-null    datetime64[ns]
 1   Symbol        218 non-null    object        
 2   Company Name  218 non-null    object        
 3   IPO Price     218 non-null    object        
 4   Current       218 non-null    object        
 5   Return        218 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.3+ KB


In [53]:
df_ipos.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.91,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.95,-19.97%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.62,-84.45%


In [54]:
df_filtered_ipos = df_ipos[(df_ipos['IPO Date'] < '2024-03-01') & (df_ipos['Symbol'] != 'RYZB')]

In [55]:
df_filtered_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    object        
 4   Current       184 non-null    object        
 5   Return        184 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.1+ KB


In [57]:
lista = []
for idx, val in df_filtered_ipos['Symbol'].items():
    if val in ['IBAC', 'PTHR', 'BKHA']:
        lista.append(val+'U')
    elif val == 'SBXC':
        lista.append(val+'-UN')
    else:
        lista.append(val)

In [58]:
lista

['IROH',
 'LGCB',
 'ZKH',
 'BAYA',
 'INHD',
 'AFJK',
 'GSIW',
 'FEBO',
 'CLBR',
 'ELAB',
 'RR',
 'DDC',
 'SHIM',
 'GLAC',
 'SGN',
 'HG',
 'CRGX',
 'ANSC',
 'AITR',
 'GVH',
 'LXEO',
 'PAPL',
 'ATGL',
 'MNR',
 'WBUY',
 'NCL',
 'BIRK',
 'GMM',
 'PMEC',
 'LRHC',
 'GPAK',
 'SPKL',
 'QETA',
 'MSS',
 'ANL',
 'SYRA',
 'VSME',
 'LRE',
 'TURB',
 'MDBH',
 'KVYO',
 'CART',
 'DTCK',
 'NMRA',
 'ARM',
 'SPPL',
 'NWGL',
 'SWIN',
 'IVP',
 'NNAG',
 'SRM',
 'SPGC',
 'LQR',
 'NRXS',
 'FTEL',
 'MIRA',
 'PXDT',
 'HRYU',
 'CTNT',
 'SRFM',
 'PRZO',
 'HYAC',
 'KVAC',
 'JNVR',
 'ELWS',
 'WRNT',
 'TSBX',
 'ODD',
 'APGE',
 'NETD',
 'SGMT',
 'BOWN',
 'SXTP',
 'PWM',
 'VTMX',
 'INTS',
 'SVV',
 'KGS',
 'FIHL',
 'GENK',
 'BUJA',
 'BOF',
 'AZTR',
 'CAVA',
 'ESHA',
 'ATMU',
 'ATS',
 'IPXX',
 'CWD',
 'SGE',
 'SLRN',
 'ALCY',
 'KVUE',
 'GODN',
 'TRNR',
 'AACT',
 'JYD',
 'USGO',
 'UCAR',
 'WLGS',
 'TPET',
 'TCJH',
 'GDTC',
 'VCIG',
 'GDHG',
 'ARBB',
 'ISPR',
 'MGIH',
 'MWG',
 'HSHP',
 'SFWL',
 'SYT',
 'HKIT',
 'CHSN',
 'T

In [59]:
ticker_df = []

for ticker in lista:
  df_ticker = yf.download(tickers = ticker,
                        period = "max",
                        interval = "1d")
  df_ticker['Symbol'] = ticker
  df_ticker["adj_close_minus_1"] = df_ticker['Adj Close'].shift(-1)
  df_ticker["adj_close_growth_1d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_1"] - 1
  #df_ticker = df_ticker.drop("adj_close_minus_1", axis=1)
  df_ticker["adj_close_minus_2"] = df_ticker['Adj Close'].shift(-2)
  df_ticker["adj_close_growth_2d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_2"] - 1
  df_ticker["adj_close_minus_3"] = df_ticker['Adj Close'].shift(-3)
  df_ticker["adj_close_growth_3d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_3"] - 1
  df_ticker["adj_close_minus_4"] = df_ticker['Adj Close'].shift(-4)
  df_ticker["adj_close_growth_4d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_4"] - 1
  df_ticker["adj_close_minus_5"] = df_ticker['Adj Close'].shift(-5)
  df_ticker["adj_close_growth_5d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_5"] - 1
  df_ticker["adj_close_minus_6"] = df_ticker['Adj Close'].shift(-6)
  df_ticker["adj_close_growth_6d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_6"] - 1
  df_ticker["adj_close_minus_7"] = df_ticker['Adj Close'].shift(-7)
  df_ticker["adj_close_growth_7d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_7"] - 1
  df_ticker["adj_close_minus_8"] = df_ticker['Adj Close'].shift(-8)
  df_ticker["adj_close_growth_8d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_8"] - 1
  df_ticker["adj_close_minus_9"] = df_ticker['Adj Close'].shift(-9)
  df_ticker["adj_close_growth_9d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_9"] - 1
  df_ticker["adj_close_minus_10"] = df_ticker['Adj Close'].shift(-10)
  df_ticker["adj_close_growth_10d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_10"] - 1
  df_ticker["adj_close_minus_11"] = df_ticker['Adj Close'].shift(-11)
  df_ticker["adj_close_growth_11d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_11"] - 1
  df_ticker["adj_close_minus_12"] = df_ticker['Adj Close'].shift(-12)
  df_ticker["adj_close_growth_12d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_12"] - 1
  df_ticker["adj_close_minus_13"] = df_ticker['Adj Close'].shift(-13)
  df_ticker["adj_close_growth_13d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_13"] - 1
  df_ticker["adj_close_minus_14"] = df_ticker['Adj Close'].shift(-14)
  df_ticker["adj_close_growth_14d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_14"] - 1
  df_ticker["adj_close_minus_15"] = df_ticker['Adj Close'].shift(-15)
  df_ticker["adj_close_growth_15d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_15"] - 1
  df_ticker["adj_close_minus_16"] = df_ticker['Adj Close'].shift(-16)
  df_ticker["adj_close_growth_16d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_16"] - 1
  #df_ticker = df_ticker.drop("adj_close_minus_1", axis=1)
  df_ticker["adj_close_minus_17"] = df_ticker['Adj Close'].shift(-17)
  df_ticker["adj_close_growth_17d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_17"] - 1
  df_ticker["adj_close_minus_18"] = df_ticker['Adj Close'].shift(-18)
  df_ticker["adj_close_growth_18d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_18"] - 1
  df_ticker["adj_close_minus_19"] = df_ticker['Adj Close'].shift(-19)
  df_ticker["adj_close_growth_19d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_19"] - 1
  df_ticker["adj_close_minus_20"] = df_ticker['Adj Close'].shift(-20)
  df_ticker["adj_close_growth_20d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_20"] - 1
  df_ticker["adj_close_minus_21"] = df_ticker['Adj Close'].shift(-21)
  df_ticker["adj_close_growth_21d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_21"] - 1
  df_ticker["adj_close_minus_22"] = df_ticker['Adj Close'].shift(-22)
  df_ticker["adj_close_growth_22d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_22"] - 1
  df_ticker["adj_close_minus_23"] = df_ticker['Adj Close'].shift(-23)
  df_ticker["adj_close_growth_23d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_23"] - 1
  df_ticker["adj_close_minus_24"] = df_ticker['Adj Close'].shift(-24)
  df_ticker["adj_close_growth_24d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_24"] - 1
  df_ticker["adj_close_minus_25"] = df_ticker['Adj Close'].shift(-25)
  df_ticker["adj_close_growth_25d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_25"] - 1
  df_ticker["adj_close_minus_26"] = df_ticker['Adj Close'].shift(-26)
  df_ticker["adj_close_growth_26d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_26"] - 1
  df_ticker["adj_close_minus_27"] = df_ticker['Adj Close'].shift(-27)
  df_ticker["adj_close_growth_27d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_27"] - 1
  df_ticker["adj_close_minus_28"] = df_ticker['Adj Close'].shift(-28)
  df_ticker["adj_close_growth_28d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_28"] - 1
  df_ticker["adj_close_minus_29"] = df_ticker['Adj Close'].shift(-29)
  df_ticker["adj_close_growth_29d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_14"] - 1
  df_ticker["adj_close_minus_30"] = df_ticker['Adj Close'].shift(-30)
  df_ticker["adj_close_growth_30d"] = df_ticker['Adj Close']/df_ticker["adj_close_minus_30"] - 1
  ticker_df.append(df_ticker)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [60]:
df_ipos_strat = pd.concat(ticker_df, axis=0, ignore_index=False)

In [61]:
df_ipos_strat

,Open,High,Low,Close,Adj Close,Volume,Symbol,adj_close_minus_1,adj_close_growth_1d,adj_close_minus_2,...,adj_close_minus_26,adj_close_growth_26d,adj_close_minus_27,adj_close_growth_27d,adj_close_minus_28,adj_close_growth_28d,adj_close_minus_29,adj_close_growth_29d,adj_close_minus_30,adj_close_growth_30d
Date,,,,,,,,,,,,,,,,,,,,,
2024-02-16,10.050,10.050,10.010,10.010,10.010,16700,IROH,10.020,-0.000998,10.015,...,9.980,0.003006,9.985,0.002504,9.980,0.003006,9.990,0.004012,9.990,0.002002
2024-02-20,10.020,10.020,10.020,10.020,10.020,5200,IROH,10.015,0.000499,10.020,...,9.985,0.003505,9.980,0.004008,9.990,0.003003,9.990,0.004008,10.025,-0.000499
2024-02-21,10.020,10.020,10.015,10.015,10.015,98600,IROH,10.020,-0.000499,10.010,...,9.980,0.003507,9.990,0.002503,9.990,0.002503,10.025,0.003507,10.017,-0.000200
2024-02-22,10.020,10.020,10.020,10.020,10.020,5600,IROH,10.010,0.000999,10.020,...,9.990,0.003003,9.990,0.003003,10.025,-0.000499,10.017,0.004008,10.013,0.000699
2024-02-23,10.020,10.020,10.010,10.010,10.010,14800,IROH,10.020,-0.000998,10.010,...,9.990,0.002002,10.025,-0.001496,10.017,-0.000699,10.013,0.003006,10.010,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.710,0.728,0.661,0.678,0.678,55500,ROMA,0.711,-0.046414,0.719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-02,0.693,0.721,0.658,0.711,0.711,202900,ROMA,0.719,-0.011127,0.696,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05-03,0.750,0.777,0.670,0.719,0.719,1066800,ROMA,0.696,0.033046,0.750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_ipos_strat.describe()

,Open,High,Low,Close,Adj Close,Volume,adj_close_minus_1,adj_close_growth_1d,adj_close_minus_2,adj_close_growth_2d,...,adj_close_minus_26,adj_close_growth_26d,adj_close_minus_27,adj_close_growth_27d,adj_close_minus_28,adj_close_growth_28d,adj_close_minus_29,adj_close_growth_29d,adj_close_minus_30,adj_close_growth_30d
count,37770.000000,37770.000000,37770.000000,37770.000000,37770.000000,3.777000e+04,37586.000000,37586.000000,37402.000000,37402.000000,...,32994.000000,32994.000000,32811.000000,32811.000000,32628.000000,32628.000000,32446.000000,35194.000000,32264.000000,32264.000000
mean,12.515059,13.139440,12.045827,12.485686,12.456813,6.385625e+05,12.336431,0.009279,12.310453,0.024988,...,12.098707,0.494959,12.095100,0.516772,12.091948,0.538639,12.088524,0.248963,12.071265,0.574628
std,38.123264,56.866338,32.879304,40.746515,40.735089,5.008113e+06,34.216165,0.226304,34.001322,1.471897,...,33.619309,10.037492,33.647051,10.269421,33.681162,10.498767,33.719379,6.946977,33.542102,10.841248
min,0.002000,0.002000,0.002000,0.002000,0.002000,0.000000e+00,0.002000,-0.853521,0.002000,-0.856555,...,0.002000,-0.933445,0.002000,-0.932351,0.002000,-0.949123,0.002000,-0.911746,0.002000,-0.963233
25%,1.890000,1.990000,1.780000,1.870000,1.870000,5.500000e+03,1.860000,-0.014510,1.850000,-0.019655,...,1.710000,-0.056048,1.710000,-0.058249,1.700000,-0.059778,1.700000,-0.044246,1.690000,-0.062385
50%,8.100000,8.362000,7.800000,8.060000,8.060000,4.940000e+04,8.055000,0.000000,8.060000,0.000000,...,7.895000,0.004864,7.881000,0.005160,7.875389,0.005366,7.860427,0.000988,7.850000,0.005958
75%,12.140000,12.460000,11.660000,12.100000,12.090993,2.213750e+05,12.085750,0.022820,12.084750,0.034483,...,12.000000,0.226657,12.000000,0.231029,12.000000,0.237913,12.000000,0.132317,12.000000,0.250000
max,3069.000000,7500.000000,875.000000,4318.000000,4318.000000,3.723413e+08,901.000000,33.875000,901.000000,278.000001,...,901.000000,365.499973,901.000000,365.499973,901.000000,365.499973,901.000000,365.499973,901.000000,365.499973


In [63]:
adj25percent, adj50percent, adj75percent = {}, {}, {}
desc = df_ipos_strat.describe().to_dict()
for i in desc.items():
  if i[0].startswith('adj_close_growth'):
    adj25percent[i[0]] = desc[i[0]]['25%']
    adj50percent[i[0]] = desc[i[0]]['50%']
    adj75percent[i[0]] = desc[i[0]]['75%']

In [64]:
def max_value_key(dict):
  max_value_key = max(dict, key=dict.get)
  max_value = dict[max_value_key]
  return (max_value_key, max_value)

In [68]:
(key25, max25value), (key50, max50value), (key75, max75value)  = max_value_key(adj25percent), max_value_key(adj50percent), max_value_key(adj75percent)
maxStrat = [key25, key50, key75][np.argmax([max25value, max50value, max75value])]
f'Best strategy is {maxStrat}'

'Best strategy is adj_close_growth_30d'

### Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**


Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks 
for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

`US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']`

`EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']`

`INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']`

`LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS`
<br/>

Now let's add the top 12-22 stocks (as of end-April 2024):
<br/>

`NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']`

`NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']`

`NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']`

`LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA`

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate  `growth_7d` for every stock and every day.
Get the average daily `growth_7d` for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:
| Date   |      ticker_category      |  growth_7d |
|----------|:-------------:|------:|
| 2014-01-01 |  LARGE | 1.011684 |
| 2014-01-01 |   LARGEST   |   1.011797 |

On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [71]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [72]:
def get_7d_growth_df(df:pd.DataFrame, i)->pd.DataFrame:
  df['growth_'+ str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
  GROWTH_KEYS = [k for k in df.keys() if k.startswith('growth')]
  return df[GROWTH_KEYS]

In [78]:
# Largest Stocks
largest_stock_df = []
i = 7
for ticker in LARGEST_STOCKS:
  df_ticker = yf.download(tickers = ticker,
                        period = "max",
                        interval = "1d")
  df_ticker['Symbol'] = ticker
  df_copy = df_ticker.copy()
  df_filtered = df_copy.loc[df_copy.index >= '2014-01-01']
  df = df_filtered.loc[df_filtered.index <= '2023-12-31']
  df['growth_'+ str(i)+'d'] = df['Adj Close'] / df['Adj Close'].shift(i)
  largest_stock_df.append(df)
df_largest = pd.concat(largest_stock_df, axis=0, ignore_index=False)
df_largest

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Open,High,Low,Close,Adj Close,Volume,Symbol,growth_7d
Date,,,,,,,,
2014-01-02,37.349998,37.400002,37.099998,37.160000,31.233059,30632200,MSFT,NaN
2014-01-03,37.200001,37.220001,36.599998,36.910000,31.022924,31134800,MSFT,NaN
2014-01-06,36.849998,36.889999,36.110001,36.130001,30.367357,43603700,MSFT,NaN
2014-01-07,36.330002,36.490002,36.209999,36.410000,30.602678,35802800,MSFT,NaN
2014-01-08,36.000000,36.139999,35.580002,35.759998,30.056356,59971700,MSFT,NaN
...,...,...,...,...,...,...,...,...
2023-12-22,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707,LT.NS,1.022987
2023-12-26,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263,LT.NS,1.016588
2023-12-27,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266,LT.NS,1.016055


In [79]:
large_stock_df = []
i = 7
for ticker in LARGE_STOCKS:
    df_stocks = yf.download(tickers = ticker,
                            period = "max",
                            interval = "1d")
    df_stocks['Symbol'] = ticker
    df_copy_stocks = df_stocks.copy()
    df_f_stocks = df_copy_stocks.loc[df_copy_stocks.index >= '2014-01-01']
    df_l = df_f_stocks.loc[df_f_stocks.index <= '2023-12-31']
    df_l['growth_'+ str(i)+'d'] = df_l['Adj Close'] / df_l['Adj Close'].shift(i)
    large_stock_df.append(df_l)
df_large = pd.concat(large_stock_df, axis=0, ignore_index=False)
df_large


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Open,High,Low,Close,Adj Close,Volume,Symbol,growth_7d
Date,,,,,,,,
2019-09-11,34.868782,35.511101,33.207928,34.038357,33.763882,14787563,PRX.AS,NaN
2019-09-12,34.501743,34.524685,32.978527,33.148285,32.880985,5842906,PRX.AS,NaN
2019-09-13,33.171223,34.042942,32.804184,33.905304,33.631897,5068221,PRX.AS,NaN
2019-09-16,33.515324,33.721783,32.340797,32.340797,32.080013,49689045,PRX.AS,NaN
2019-09-17,32.758305,33.033585,31.574600,31.794825,31.538437,10523710,PRX.AS,NaN
...,...,...,...,...,...,...,...,...
2023-12-22,3580.000000,3638.449951,3560.550049,3627.350098,3627.350098,777099,TITAN.NS,1.006940
2023-12-26,3635.000000,3665.000000,3623.449951,3656.699951,3656.699951,526101,TITAN.NS,1.018182
2023-12-27,3668.000000,3695.000000,3645.000000,3689.250000,3689.250000,666625,TITAN.NS,1.024635


In [80]:
df_large_pivot = pd.pivot_table(df_large, values='growth_7d', index='Date', aggfunc = np.mean)
df_large_pivot

,growth_7d
Date,
2014-01-10,0.976317
2014-01-13,0.997588
2014-01-14,1.006721
2014-01-15,1.014193
2014-01-16,1.013045
...,...
2023-12-22,1.003014
2023-12-26,1.013505
2023-12-27,1.002167


In [81]:
df_largest_pivot = pd.pivot_table(df_largest, values='growth_7d', index='Date', aggfunc = np.mean)
df_largest_pivot

,growth_7d
Date,
2014-01-10,0.967215
2014-01-13,0.993122
2014-01-14,0.999657
2014-01-15,1.012536
2014-01-16,1.007409
...,...
2023-12-22,1.013788
2023-12-26,1.014900
2023-12-27,1.014155


In [82]:

df_all_pivot = pd.merge(df_large_pivot, df_largest_pivot, how='left', left_index=True, right_index=True)
df_all_pivot

,growth_7d_x,growth_7d_y
Date,,
2014-01-10,0.976317,0.967215
2014-01-13,0.997588,0.993122
2014-01-14,1.006721,0.999657
2014-01-15,1.014193,1.012536
2014-01-16,1.013045,1.007409
...,...,...
2023-12-22,1.003014,1.013788
2023-12-26,1.013505,1.014900
2023-12-27,1.002167,1.014155


In [83]:
df_all_pivot['large_greater_than_largest'] = (df_all_pivot['growth_7d_x'] > df_all_pivot['growth_7d_y']).astype(int)

In [84]:
df_all_pivot

,growth_7d_x,growth_7d_y,large_greater_than_largest
Date,,,
2014-01-10,0.976317,0.967215,1
2014-01-13,0.997588,0.993122,1
2014-01-14,1.006721,0.999657,1
2014-01-15,1.014193,1.012536,1
2014-01-16,1.013045,1.007409,1
...,...,...,...
2023-12-22,1.003014,1.013788,0
2023-12-26,1.013505,1.014900,0
2023-12-27,1.002167,1.014155,0


In [86]:
round(df_all_pivot['large_greater_than_largest'].sum()/2595*100, 2)

47.28

Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [3]:
merged_df_with_tech_ind = pd.read_parquet('stocks_df_combined_trunc_2014_2023.parquet.brotli')

In [12]:
merged_df_with_tech_ind.columns[:50]

Index(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d',
       'growth_7d', 'growth_30d', 'growth_90d', 'growth_365d',
       'growth_future_5d', 'SMA10', 'SMA20', 'growing_moving_average',
       'high_minus_low_relative', 'volatility', 'is_positive_growth_5d_future',
       'ticker_type', 'index_x', 'adx', 'adxr', 'apo', 'aroon_1', 'aroon_2',
       'aroonosc', 'bop', 'cci', 'cmo', 'dx', 'macd', 'macdsignal', 'macdhist',
       'macd_ext', 'macdsignal_ext', 'macdhist_ext', 'macd_fix',
       'macdsignal_fix', 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di',
       'dm'],
      dtype='object')

In [15]:
filtered_df = merged_df_with_tech_ind[
    (merged_df_with_tech_ind.Date >= '2014-01-01') &
    (merged_df_with_tech_ind.Date <= '2023-12-31')
]

In [16]:
filtered_df

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5338,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
5339,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
5340,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
5341,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [25]:
!pip install ta-lib

  Using cached TA-Lib-0.4.28.tar.gz (357 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build ta-lib


  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [14 lines of output]
      <string>:77: UserWarning: Cannot find ta-lib library, installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\talib
      copying talib\abstract.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\deprecated.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\stream.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\__init__.py -> build\lib.win-amd64-cpython-312\talib
      running build_ext
      building 'talib._ta_lib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error origi

In [24]:
import talib

ModuleNotFoundError: No module named 'talib'

In [ ]:
# Test
talib_momentum_cci = talib.CCI(df.High.values, df.Low.values, df.Close.values, timeperiod=14)

As I have error trying to install talib on windows and wsl2. I choose for code this on colab.

Link: [https://colab.research.google.com/drive/1WZm1YrTNIkWiQI8fWxrYgl4AkTKxtqq0?usp=sharing](https://colab.research.google.com/drive/1WZm1YrTNIkWiQI8fWxrYgl4AkTKxtqq0?usp=sharing)

### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

E.g. (some ideas) Do you want to focus on the specific vertical? Do you want to build a smart comparison vs. existing stocks on the market? Or you just will want to get some features (which features?) like total number of people in a company to find a segment of "successful" IPOs?